In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
#unzip Autopilot folder as data is inside there
!cp  /content/drive/Hackthon/McKinsey_Analytics/* .


In [4]:
!ls 

01_McKinsey_Analytics.ipynb
02_McKinsey_Analytics.ipynb
03_McKinsey_Analytics.ipynb
1000_double_half_adam_simple_Mckiney_model.h5
100_unbalance_adam_simple_Mckiney_model.h5
200_double_half_adam_simple_Mckiney_model.h5
200_unbalance_adam_simple_Mckiney_model.h5
AUC_simple_Mckiney_model.h5
datalab
drive
feature_stand_100_lr_0001_07_drop_adam_cv_01780_model.h5
feature_stand_100_lr_0001_adam_simple_01784_Mckiney_model.h5
feature_stand_100_lr_001_07_drop_simple_Mckiney_model.h5
feature_stand_100_unbalance_adam_simple_Mckiney_model.h5
feature_stand_150_lr_0001_adam_simple_0_1778_Mckiney_model.h5
feature_stand_200_lr_0001_adamax_simple_Mckiney_model.h5
feature_stand_200_unbalance_adam_simple_Mckiney_model.h5
feature_stand_300_unbalance_adam_simple_Mckiney_model.h5
sample_submission_sLex1ul.csv
simple_Mckiney_model.h5
test_66516Ee.csv
train_ZoGVYWq.csv
Untitled0.ipynb



<h1>3. Exploratory Data Analysis</h1>
<h3>3.1. Import All required Classed first</h3>


In [5]:
import numpy as np
import pandas as pd
import os, shutil
from keras.models import Model
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense,Input,ELU
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from scipy import pi
import matplotlib.pyplot as plt
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.regularizers import l2
from keras.optimizers import Adam

from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras import layers
from keras.layers import Input, Dense, Dropout,LeakyReLU,BatchNormalization
from keras import backend as K
from sklearn.model_selection import train_test_split
import bisect
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from keras.models import load_model
from keras import regularizers
from keras.optimizers import Adam
from numpy import argmax

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
Data = pd.read_csv("test_66516Ee.csv")

In [4]:
print(Data.shape)
print(Data.columns)

(34224, 12)
Index(['id', 'perc_premium_paid_by_cash_credit', 'age_in_days', 'Income',
       'Count_3-6_months_late', 'Count_6-12_months_late',
       'Count_more_than_12_months_late', 'application_underwriting_score',
       'no_of_premiums_paid', 'sourcing_channel', 'residence_area_type',
       'premium'],
      dtype='object')


In [5]:
print(Data.describe())

                  id  perc_premium_paid_by_cash_credit   age_in_days  \
count   34224.000000                      34224.000000  34224.000000   
mean    56739.956376                          0.314457  18824.215346   
std     32935.402647                          0.334059   5246.525604   
min         1.000000                          0.000000   7671.000000   
25%     28254.750000                          0.034000  14972.000000   
50%     56488.000000                          0.169000  18623.000000   
75%     85386.250000                          0.540000  22636.000000   
max    114077.000000                          1.000000  35785.000000   

             Income  Count_3-6_months_late  Count_6-12_months_late  \
count  3.422400e+04           34193.000000            34193.000000   
mean   2.028201e+05               0.238733                0.080718   
std    2.702536e+05               0.686162                0.454634   
min    2.403000e+04               0.000000                0.000000   
2

In [6]:
values = {"Count_3-6_months_late":0,"Count_6-12_months_late":0,"Count_more_than_12_months_late":0,"application_underwriting_score":Data["application_underwriting_score"].mean()}
Data.fillna(value=values,inplace=True)
Data.shape

(34224, 12)

In [7]:
Data.isna().values.any()


False

In [0]:
#Combining months late feature
Data['count_month_late'] = 20*Data["Count_3-6_months_late"] + 30*Data["Count_6-12_months_late"]+50*Data["Count_more_than_12_months_late"]

Feature Standarization and Normalization

In [0]:
def min_max_norm(dat_set):
  
  max = dat_set["perc_premium_paid_by_cash_credit"].max()
  dat_set["perc_premium_paid_by_cash_credit_norm"] =  round(dat_set["perc_premium_paid_by_cash_credit"]/max,5)
  
  max = dat_set["Income"].max()
  dat_set["Income_norm"] =  round(dat_set["Income"]/max,5)
  
  max = dat_set["Count_3-6_months_late"].max()
  dat_set["Count_3-6_months_late_norm"] =  round(dat_set["Count_3-6_months_late"]/max,5)
  
  max = dat_set["Count_6-12_months_late"].max()
  dat_set["Count_6-12_months_late_norm"] =  round(dat_set["Count_6-12_months_late"]/max,5)
  
  max = dat_set["Count_more_than_12_months_late"].max()
  dat_set["Count_more_than_12_months_late_norm"] =  round(dat_set["Count_more_than_12_months_late"]/max,5)
  
  max = dat_set["application_underwriting_score"].max()
  dat_set["application_underwriting_score_norm"] =  round(dat_set["application_underwriting_score"]/max,5)

  max = dat_set["no_of_premiums_paid"].max()
  dat_set["no_of_premiums_paid_norm"] =  round(dat_set["no_of_premiums_paid"]/max,5)
  
  max = dat_set["premium"].max()
  dat_set["premium_norm"] =  round(dat_set["premium"]/max,5)
  
  max = dat_set["count_month_late"].max()
  dat_set["count_month_late_norm"] =  round(dat_set["count_month_late"]/max,5)
  return dat_set
Data = min_max_norm(Data)

In [0]:
data_set = Data.loc[:,["perc_premium_paid_by_cash_credit", "Income_norm", "application_underwriting_score_norm", "no_of_premiums_paid_norm", "premium_norm"]]

scaler = StandardScaler()
data_set_st = scaler.fit_transform(data_set)

Dealing with Categorical data

'age_in_days','sourcing_channel', 'residence_area_type',

In [0]:
def age_group(age):
  age=age/(365)
  if age < 25:
    return  25
  if age < 30:
    return 30
  if age < 35:
    return 35
  if age < 40:
    return 40
  if age < 60:
    return 60
  if age < 65:
    return 65
  if age < 70:
    return 70
  if age < 75:
    return 75
  if age < 80:
    return 80
  else:
    return 100
  
Data["age_in_years_group"] = Data["age_in_days"].map(age_group)

In [12]:
#check whether age_group convertion as expected
print(Data["age_in_years_group"].head(10))
print(Data["age_in_days"].head(10)/(365))

0    80
1    70
2    60
3    60
4    30
5    60
6    60
7    60
8    65
9    65
Name: age_in_years_group, dtype: int64
0    75.024658
1    65.027397
2    47.041096
3    44.021918
4    29.016438
5    44.013699
6    47.032877
7    55.027397
8    63.035616
9    60.016438
Name: age_in_days, dtype: float64


In [0]:
label_encoder_ag = LabelEncoder()
label_encoder_sc = LabelEncoder()
label_encoder_ar = LabelEncoder()
label_encoder_out = LabelEncoder()
ag_cat=to_categorical(label_encoder_ag.fit_transform(Data["age_in_years_group"]))
sc_cat=to_categorical(label_encoder_sc.fit_transform(Data["sourcing_channel"]))
ar_cat=to_categorical(label_encoder_ar.fit_transform(Data["residence_area_type"]))
#out_cat=to_categorical(label_encoder_out.fit_transform(Data["renewal"]))


In [14]:
print(ag_cat.shape)
print(sc_cat.shape)
print(ar_cat.shape)
#print(out_cat.shape)

(34224, 10)
(34224, 5)
(34224, 2)


In [15]:
# stacking all features categorical

data_cat = np.hstack((ag_cat,sc_cat,ar_cat))
print(data_cat.shape)

(34224, 17)


In [16]:
Sub_Data = Data.loc[:,["perc_premium_paid_by_cash_credit_norm", 
                       "Income_norm",
                       "Count_3-6_months_late_norm",
                       "Count_6-12_months_late_norm",
                       "Count_more_than_12_months_late_norm",
                       "application_underwriting_score_norm",
                       "no_of_premiums_paid_norm",
                       "premium_norm",
                       "count_month_late_norm"]]

print(Sub_Data.shape)
Sub_Data_np = np.asarray(Sub_Data)
print(Sub_Data_np.shape)

(34224, 9)
(34224, 9)


In [17]:
norm_Sub_Data = Data.loc[:,["Count_3-6_months_late_norm",
                        "Count_6-12_months_late_norm",
                       "Count_more_than_12_months_late_norm"]]

print(norm_Sub_Data.shape)
norm_Sub_Data_np = np.asarray(norm_Sub_Data)
print(norm_Sub_Data_np.shape)

(34224, 3)
(34224, 3)


In [18]:
#data_xs = np.hstack((Sub_Data_np,data_cat))
data_xs = np.hstack((data_set_st,norm_Sub_Data_np,data_cat))
print(data_xs.shape)


(34224, 25)


In [19]:
print(data_xs.shape[1])

25


In [0]:
class_weight = {0:1,1:1} #0.8405
#class_weight = {0:8,1:0.5} # 1000 ep - auc 8916
class_weight = {0:4,1:0.25} # 1000 ep - auc 8916

In [0]:
from sklearn import metrics
import tensorflow as tf

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [0]:
model = load_model("feature_stand_150_lr_0001_adam_simple_0_1778_Mckiney_model.h5",custom_objects={"auc":auc})

In [0]:
y_pre = model.predict(data_xs)

In [56]:
print(y_pre.max())
print(y_pre.min())
print(y_pre.mean())

0.9985923
0.10946207
0.9266927


In [57]:
np.percentile(y_pre,[1,5,10,14,25])


array([0.39107529, 0.64007655, 0.81254246, 0.86500297, 0.92842989])

In [0]:
y_round = list(map(lambda x:round(float(x),5),y_pre[:]))

In [0]:
Data["renewal"] = y_round

In [60]:
np.percentile(Data["renewal"],[1,5,7,8,9,10,14,25])

array([0.3910752, 0.6400735, 0.7340544, 0.7685068, 0.7930635, 0.812543 ,
       0.8650022, 0.92843  ])

In [61]:
Data.head()

,id,perc_premium_paid_by_cash_credit,age_in_days,Income,Count_3-6_months_late,Count_6-12_months_late,Count_more_than_12_months_late,application_underwriting_score,no_of_premiums_paid,sourcing_channel,...,Count_3-6_months_late_norm,Count_6-12_months_late_norm,Count_more_than_12_months_late_norm,application_underwriting_score_norm,no_of_premiums_paid_norm,premium_norm,count_month_late_norm,age_in_years_group,renewal,incentives
0,649,0.001,27384,51150,0.0,0.0,0.0,99.890000,7,A,...,0.00000,0.0,0.0,1.00000,0.11864,0.055,0.00000,80,0.99395,0
1,81136,0.124,23735,285140,0.0,0.0,0.0,98.930000,19,A,...,0.00000,0.0,0.0,0.99039,0.32203,0.195,0.00000,70,0.98780,0
2,70762,1.000,17170,186030,0.0,0.0,0.0,99.061898,2,B,...,0.00000,0.0,0.0,0.99171,0.03390,0.195,0.00000,60,0.91988,150
3,53935,0.198,16068,123540,0.0,0.0,0.0,99.000000,11,B,...,0.00000,0.0,0.0,0.99109,0.18644,0.090,0.00000,60,0.97731,0
4,15476,0.041,10591,200020,1.0,0.0,0.0,99.170000,14,A,...,0.08333,0.0,0.0,0.99279,0.23729,0.160,0.03922,30,0.92261,100


In [0]:
import math
def calculate_per_improve(incentive):
  extra_hr = 10*(1- math.exp(-(incentive/400)))
  per_imp = 20*(1- math.exp(-(extra_hr/5)))
  return round(per_imp/100,4)
 
calculate_per_improve(1000)

def calculate_revenue_gain(p_benchmark,per_improve,premium_on_policy,incentive):
  return ((p_benchmark + p_benchmark*per_improve)*premium_on_policy-incentive)




In [63]:
incentive_list =  [i for i in range(50,1600,50)]
per_improve_list = list(map(calculate_per_improve,incentive_list))
print(per_improve_list)
def predict_incentive(p_benchmark,premium_on_policy):
    revenue_gain_no_incentive=calculate_revenue_gain(p_benchmark,0,premium_on_policy,0)
    best_incentive = 0
    for i,p in zip(incentive_list,per_improve_list):
        revenue_gain = calculate_revenue_gain(p_benchmark,p,premium_on_policy,i)
        if revenue_gain_no_incentive < revenue_gain:
            if (p_benchmark + p_benchmark*p)<1:
              revenue_gain_no_incentive =revenue_gain
              best_incentive = i
    return best_incentive

[0.0419, 0.0715, 0.093, 0.109, 0.121, 0.1304, 0.1377, 0.1435, 0.1482, 0.152, 0.1551, 0.1577, 0.1599, 0.1617, 0.1632, 0.1645, 0.1656, 0.1666, 0.1674, 0.1681, 0.1687, 0.1692, 0.1697, 0.1701, 0.1704, 0.1708, 0.171, 0.1712, 0.1715, 0.1716, 0.1718]


In [0]:
Data["incentives"] = list(map(predict_incentive,Data.renewal,Data.premium))


In [0]:
out_data = pd.DataFrame()
out_data["id"] = Data.id
out_data["renewal"] = Data["renewal"]
out_data['incentives'] = Data["incentives"]

In [66]:
out_data.head()

,id,renewal,incentives
0,649,0.99395,0
1,81136,0.98780,0
2,70762,0.91988,100
3,53935,0.97731,0
4,15476,0.92261,100


In [0]:
out_data.to_csv("feature_stand_150_lr_0001_adam_simple_0_1778_Mckiney_model.csv",index=False)

In [0]:
from google.colab import files
files.download("feature_stand_150_lr_0001_adam_simple_0_1778_Mckiney_model.csv")